# Requests

- requests: 서버에 request 요청을 하고 response를 받는 패키지
- [http://docs.python-requests.org/en/master/](http://docs.python-requests.org/en/master/)
- mac은 설치 필요: `pip3 install requests`

In [25]:
import requests
import pickle

## 1. 네이버 주식 데이터 가져오기
- api를 사용해 json 파싱
- 네이버 주식 페이지(모바일)에서 주식 데이터를 가져와 pandas dataframe으로 만들기
- [http://m.stock.naver.com](http://m.stock.naver.com)
- 증권-국내-코스피: http://m.stock.naver.com/sise/siseList.nhn?menu=market_sum&sosok=0
- 개발자도구에서 request url을 복사해오기
    - https://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=20&page=2

### 1.1 make_url 함수
- page_size와 page를 parameter로 받아 url을 만드는 함수

In [14]:
def make_url(page_size=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
menu=market_sum&sosok=0&pageSize="+str(page_size)+"&page="+str(page)


make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

### 1.2 get_data 함수
- url을 parameter로 받아 pandas dataframe으로 만들어주는 함수

In [22]:
def get_data(url):
    """
    URL을 받으면 json으로 데이터를 가져와서 pandas dataframe 로 만들어주는 함수
    """
    response = requests.get(url)   # response라는 obj. 안에 결과 데이터를 받아옴
    json_info = response.json()    # json함수: 문자열 데이터를 json 형태로 parsing
    companies = json_info["result"]["itemList"]
    df = pd.DataFrame(columns = ["종목","시세","전일비","등락율","시가총액","거래량"])
    for company in companies:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"]
        }
    return df   

In [23]:
url = make_url(100, 1)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
95,KODEX 레버리지,14885,-170,-1.13,27120,5641593
96,대한항공,28100,0,0.00,26651,385938
97,TIGER 200,30135,-175,-0.58,26428,861828
98,OCI,110000,3000,2.80,26234,156276
99,두산,136500,-1500,-1.09,26205,54640


In [1]:
from pandas.io.json import json_normalize

In [11]:
def get_data(url):
    
    response = requests.get(url)
    json_info = response.json()    # parsing
    companies = json_info["result"]["itemList"]
    return json_normalize(companies)

url = make_url(100, 2)
df = get_data(url)
df.tail()

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
95,645,38116,005690,-0.59,-100,10086,0,파미셀,16850,16950,5
96,420,9161,079550,-1.83,-850,10054,0,LIG넥스원,45700,46550,5
97,62,1383,226320,0.88,400,10011,0,잇츠한불,45650,45250,2
98,6966,72049,001820,2.02,1900,9969,0,삼화콘덴서,95900,94000,2
99,129,4536,001680,-0.18,-50,9875,0,대상,28500,28550,5


## 2. 공식적으로 제공되는 API 사용하기 - Dark Sky API
- 날씨 정보를 알려주는 api - [https://darksky.net/dev](https://darksky.net/dev)
    - 회원가입 필요, 하루에 1000건까지 무료로 사용 가능
    - 사용법([Docs](https://darksky.net/dev/docs))
        - Forecast Request: https://api.darksky.net/forecast/[key]/[latitude],[longitude]
        - [key]: 회원가입시 부여받은 secret key 사용
            - 요청할 때 token 값을 같이 query에 보내주어 API를 요청하는 사람이 누구인지 서비스 제공자가 판별

- mac 설치: `pip3 install python-forecastio`
- window 설치: 
    - `conda install -c auto python-forecastio`
    - `conda install -c anaconda python-forecastio`
    - `conda install -c conda-forge python-forecastio` 
    - `pip3 install python-forecastio`

### 2.1 Preparation
- import package & set request url
- secret key(토큰)는 유출되어서는 안되니 pickle file로 저장해 이용

In [2]:
import forecastio

In [27]:
# FORECAST_TOKEN = "본인이 발급받은 토큰"
# pickle.dump(FORECAST_TOKEN, open("FORECAST_TOKEN.pickle", 'wb'))
FORECAST_TOKEN = pickle.load(open('FORECAST_TOKEN.pickle', 'rb'))

### 2.2 forecast 함수 사용
- 위도와 경도를 parameter로 받는 forecast 함수 생성

In [29]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["hourly"]["summary"], json_info["timezone"]

In [30]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Partly cloudy tomorrow afternoon.', 'Asia/Seoul')

### 2.3 forecastio 패키지 사용

In [31]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng, lang = 'ko')
    byHour = forecast.hourly()
    return byHour.summary

In [32]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'내일 오후동안 약간 흐림'

## 3. BeautifulSoup
- BeautifulSoup Doc (https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- 문자열로 되어있는 html 코드를 css selector나 xpath를 이용해서 특정 element의 데이터를 쉽게 가져올 수 있도록 하는 서비스 패키지
- 설치: `pip3 install bs4`

In [33]:
from bs4 import BeautifulSoup

### 3.1 네이버 검색어 순위 크롤링
- bs4 사용 : html을 특정 element를 CSS selector로 select할 수 있는 객체로 parsing
- 네이버 검색어 순위를 가져와 데이터 프레임으로 만들기
- [http://naver.com](http://naver.com)

In [34]:
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://naver.com")
    dom = BeautifulSoup(response.content, "html.parser")   #html.parser   # dom 객체
    keywords = dom.select(".ah_roll .ah_l .ah_item")   
        # CSS selector - selecting하는 방법은 다양
        # select 쓰면 list로 리턴 (요소가 하나라도) | select_one은 하나의 element만 리턴
        # len()을 찍어봐서 갯수가 맞게 나오는지 확인할 것
    # keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")  
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ah_r").text,       
            "keyword": keyword.select_one(".ah_k").text,
            # .text - element 안의 시작태그와 끝 태그 사이 내용을 가져옴
        }
    return df

In [35]:
df = naver_top20()
df.tail()

,rank,keyword
15,16,박미선
16,17,하나뿐인 내편
17,18,중국 샤먼 여행
18,19,그녀로 말할 것 같으면
19,20,토익


10초마다 실시간 검색어를 크롤링해서 리스트로 저장하기

In [36]:
import time
ls = []
for idx in range(3):
    print(idx)
    df = naver_top20()
    ls.append(df)
    time.sleep(10)

0
1
2


In [37]:
ls[2]

,rank,keyword
0,1,로또826회당첨번호
1,2,맨유 웨스트햄
2,3,그것이 알고싶다
3,4,류현진
4,5,아는형님
5,6,그녀로 말할 것 같으면
6,7,플레이어
7,8,로또825회당첨번호
8,9,맨유
9,10,epl


### 3.2 다음 검색어 순위
- 다음 검색어 순위를 가져와서 네이버 검색어 순위와 중복되는 키워드만 출력하기
- http://daum.net

#### 다음 실시간 검색어 순위 가져오기

In [51]:
def daum_top10():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://daum.net")
    dom = BeautifulSoup(response.content, "html.parser")   
    keywords = dom.select("#mArticle > div.cmain_tmp > div.section_media > div.hotissue_builtin > div.realtime_part > ol > li")
    print(len(keywords))
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text.replace("위",""),
            "keyword":keyword.select_one(".link_issue").text,
        }
    return df

In [53]:
df_daum = daum_top10()
df_daum

10


,rank,keyword
0,1,중국 샤먼
1,2,양희은
2,3,826회 로또 당첨 번호
3,4,김민정
4,5,플레이어
5,6,경리
6,7,여자배구
7,8,일본 태풍
8,9,주이
9,10,송병준


#### 네이버 실시간 검색어와 중복되는 키워드 추출하기
- 여러가지 방법이 있음

In [66]:
df_naver = df

result = [keyword for keyword in df_daum["keyword"] 
          if keyword in df_naver["keyword"].tolist()]
result

['중국 샤먼', '플레이어']

In [61]:
result = [keyword for keyword in df_daum["keyword"] 
          if df_naver["keyword"].str.contains(keyword).any()]
result

['중국 샤먼', '플레이어']

In [69]:
list(set(df_daum["keyword"]) & set(df_naver["keyword"]))

['중국 샤먼', '플레이어']

## 4. url 링크로 파일 다운로드 받는 방법
- 이미지 파일이나 영상 파일을 다운로드 받고 싶으면 이 방법 사용
- 용량이 큰 파일을 받을 때는 chunk를 이용해서 파일을 쪼개서 다운로드 
    - 쪼개서 받지 않으면 에러가 나거나 하는 경우 처음부터 다시 받아야 함

In [71]:
def download(title, download_url):
    response = requests.get(download_url, stream=True)
    size = 0
    with open(title, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): #1024 byte 단위로 쪼개서 받음
            if chunk:
                size += 1024
                f.write(chunk)
    return size            

Youtube 영상은 영상 링크에서 "www."를 빼고 "ss"를 붙이면 다운로드 받을 수 있으므로, 여기서 다운로드 링크를 가져오기

In [73]:
title = "Tiffany_TeachYou.mp4"
download_url = "https://bit.ly/2P1YDPT"
size = download(title, download_url)
round(size/1024/1024, 2)    # byte 단위를 megabyte 단위로 만들어 줌

22.06

#### 결론: 크롤링을 많이 연습해보자!!

#### 참고자료
- 패스트캠퍼스, ⟪데이터사이언스스쿨 8기⟫ 수업자료